In [1]:
import psutil
import joblib
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import torch.nn as nn
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
import gc
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('../input/practice-log-demographics/Practice_Log_Demographics (1).csv', low_memory=False)

In [3]:
FEATURES = ['chapter_label', 'sub_chapter_label','question_name','user_id.x','term','STDNT_SEX_CD', 
            'NonNativeEnglish', 'White', 'Asian','WhiteOrAsian', 'Hispanic','AfricanAmerican','OtherEthnicities',
            'NonWhiteOrAsian','STDNT_CTZN_STAT_CD', 'international','gradingType','birthYear','exclClassCumGPA',
            'Freshman', 'Junior','Sophomore','Senior','termCreditsGPA','termCreditsNoGPA','athlete_1','honorsPro',
            'LSA', 'programBusiness', 'programEngineering','programInformation', 'programOther','HSCalculusTaken', 
            'highSchoolGPA', 'majorsCount', 'minorsCount','PREV_TERM_CUM_GPA', 'classGraded', 'classHonors', 
            'Pass_Fail', 'parentsGraduateEdu',  'minorityGroup', 'q',"day's_available_flashcards", 
            'start_practice', 'end_practice','days_offset']

In [4]:
data = data[FEATURES]

In [5]:
gc.collect()

60

In [6]:
data['COVID_Graded'] = np.where(data['gradingType']=='COVID Graded', 1, 0)
data.drop(columns=['gradingType'], inplace=True)

In [7]:
data['term_value'] = [int(ele[3:]) for ele in data['term']]
data['age'] = data['term_value'] - data['birthYear']

In [8]:
data.drop(columns=['term_value', 'birthYear'], inplace=True)

In [9]:
#convert start_practice and end_practice to datatime column
data.start_practice = pd.to_datetime(data.start_practice, format='%Y-%m-%d %H:%M:%S')
data.end_practice = pd.to_datetime(data.end_practice, format='%Y-%m-%d %H:%M:%S')
#label encode term, chapter_label, question_name, sub_chapter_label columns
data['term'] = data['term'].astype('category')
data['chapter_id'] = data['chapter_label'].replace(data['chapter_label'].unique(), list(range(data['chapter_label'].nunique()))) + 1
data['sub_chapter_id'] = data['sub_chapter_label'].replace(data['sub_chapter_label'].unique(), list(range(data['sub_chapter_label'].nunique()))) + 1
data['question_id'] = data['question_name'].replace(data['question_name'].unique(), list(range(data['question_name'].nunique()))) + 1

In [10]:
data['dif'] = data.end_practice - data.start_practice
data['dif'] = data['dif'] /np.timedelta64(1, 's')
data['q'] = np.where((data['q']==5) & (data['dif'] <= 60), 1, 0)

In [11]:
data.drop(columns = ['chapter_label', 'sub_chapter_label', 'question_name'], inplace=True)

In [12]:
new_minors_count = []
for i in data['minorsCount']:
  if i == 0 or i == '0':
    new_minors_count.append(0)
  elif i == '1 Minor':
    new_minors_count.append(1)
  else:
    new_minors_count.append(2)

data['minorsCount'] = new_minors_count

In [13]:
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)

In [14]:
data['user_id.x'] = data['term'].astype(str) + data['user_id.x'].astype(str)
data['user_id.x'] = data['user_id.x'].replace(data['user_id.x'].unique(), list(range(data['user_id.x'].nunique())))

In [15]:
data['term'] =  data['term'].cat.codes

In [16]:
data['prev_time_elapsed'] = None
data['time_lag'] = None
data['time_lag'] = data['time_lag'].astype(np.float)
data['prev_time_elapsed'] = data['prev_time_elapsed'].astype(np.float)

In [17]:
unique_chapts = data['chapter_id'].unique()
from tqdm import tqdm
cnt = 0
for user in tqdm(data['user_id.x'].unique()):
        for chapter in unique_chapts:
            tmp_user = data[(data['user_id.x']==user) & (data['chapter_id']==chapter)]
            if len(tmp_user) < 1:
                continue
            tmp_time_elapsed = tmp_user.end_practice - tmp_user.start_practice
            tmp_time_elapsed = tmp_time_elapsed / np.timedelta64(1, 's')
            #shifting time elapsed by one
            #so that time_elapsed row for each question
            #would refer to the time that user took to answer
            #previous question
            tmp_time_elapsed = np.insert(np.array(tmp_time_elapsed[:-1]), 0, 0., axis=0)
            tmp_time_elapsed = np.cumsum(tmp_time_elapsed)
            indices = tmp_user.index
            start_row = indices[0]
            data['time_lag'].iloc[start_row] = 0
            for_mean = np.arange(len(tmp_user))
            for_mean[0] = 1
            time_substrahend = tmp_user.start_practice.iloc[:-1]
            time_substrahend = time_substrahend.apply(lambda a: a.timestamp())
            time_substrahend = np.array(time_substrahend)
            
            time_minuend = tmp_user.start_practice.iloc[1:]
            time_minuend = time_minuend.apply(lambda a: a.timestamp())
            time_minuend = np.array(time_minuend)

            data['prev_time_elapsed'].iloc[indices] = tmp_time_elapsed/for_mean
            data['time_lag'].iloc[indices[1:]] = time_minuend - time_substrahend

100%|██████████| 1110/1110 [05:07<00:00,  3.61it/s]


In [18]:
data['time_lag'].describe()

count     610792.0
unique    143509.0
top            0.0
freq       23878.0
Name: time_lag, dtype: float64

In [19]:
data['prev_time_elapsed'].describe()

count    610792.000000
mean        142.486562
std         761.014785
min           0.000000
25%          29.307692
50%          59.086957
75%         120.291667
max      157895.500000
Name: prev_time_elapsed, dtype: float64

In [20]:
#now we can drop end practice
data.drop(columns=['end_practice'], inplace=True)
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)
data['answered_correctly'] = data['q']
data.drop(columns='q', inplace=True)
gc.collect()

73

In [21]:
gc.collect()

20

In [22]:
## cv 0.7993

In [23]:
## epoch - 0 train_loss - 0.4103 train_auc - 0.7990 val_loss - 0.5121 val_auc - 0.7993 time=641.96s

In [24]:
data.fillna(0, inplace=True)

In [25]:
MAX_SEQ = 200
n_part = data['sub_chapter_id'].nunique() + 1
D_MODEL = 128
N_LAYER = 2
DROPOUT = 0.2

In [26]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(DROPOUT)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAINTModel(nn.Module):
    def __init__(self, n_skill, n_part, max_seq=MAX_SEQ, embed_dim= D_MODEL, elapsed_time_cat_flag = False):
        super(SAINTModel, self).__init__()

        self.n_skill = n_skill
        self.embed_dim = embed_dim
        self.n_chapter= 39
        self.n_sub_chapter = n_part
        self.elapsed_time_cat_flag = elapsed_time_cat_flag

        self.q_embedding = nn.Embedding(self.n_skill+1, embed_dim) ## exercise
        self.c_embedding = nn.Embedding(self.n_chapter+1, embed_dim) ## category
        self.sc_embedding = nn.Embedding(self.n_sub_chapter, embed_dim) ## category
        self.pos_embedding = nn.Embedding(max_seq+1, embed_dim) ## position
        self.res_embedding = nn.Embedding(2+1, embed_dim) ## response
        self.feat_embedding = nn.Linear(2, embed_dim)
    



        self.transformer = nn.Transformer(nhead=8, d_model = embed_dim, num_encoder_layers= N_LAYER, num_decoder_layers= N_LAYER, dropout = DROPOUT)

        self.dropout = nn.Dropout(DROPOUT)
        self.layer_normal = nn.LayerNorm(embed_dim) 
        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, question, chapter, schapter, response, user_features):

        device = question.device  
        ## embedding layer
        question = self.q_embedding(question)
        chapter = self.c_embedding(chapter)
        schapter = self.sc_embedding(schapter)
        pos_id = torch.arange(question.size(1)).unsqueeze(0).to(device)
        pos_id = self.pos_embedding(pos_id)
        res = self.res_embedding(response)
        user_features = self.feat_embedding(user_features)
        

        enc = pos_id + question + chapter + schapter 
        dec = pos_id + res + enc + user_features
        enc = enc.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        dec = dec.permute(1, 0, 2)
        mask = future_mask(enc.size(0)).to(device)
        att_output = self.transformer(enc, dec, src_mask=mask, tgt_mask=mask, memory_mask = mask)
        att_output = self.layer_normal(att_output)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]
        
        
        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

In [27]:
n_skill = data['question_id'].nunique() + 1

In [28]:
n_skill

1069

In [29]:
class PRACTICE_DATASET(Dataset):
    def __init__(self, data, maxlength=MAX_SEQ, test=False):
        super(PRACTICE_DATASET, self).__init__()
        self.maxlength = maxlength
        self.data = data
        self.test = test
        self.users = list()
        for user in data.index:
            self.users.append(user)
            
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, ix):
        user = self.users[ix]
        question_id, chapter, schapter, term, user_feats, labels = self.data[user]
        question_id = np.array(question_id, np.int16)
        chapter = np.array(chapter, np.int16)
        schapter = np.array(schapter, np.int16)
        user_feats = np.array(user_feats, np.float)
        labels = np.array(labels, np.int8)   
            
        term = term.values
        term_encode = [0]*7
        term_encode[term[0]] = 1
        n = len(question_id)
        
        if n > self.maxlength:
            question_id = question_id[-self.maxlength : ]
            chapter = chapter[-self.maxlength :]
            schapter = schapter[-self.maxlength: ]
            user_feats = user_feats[-self.maxlength :, :]
            labels = labels[-self.maxlength: ]
            responses = np.append(2, labels[:-1])
        else:
            question_id = np.pad(question_id, (self.maxlength - n, 0))
            chapter = np.pad(chapter, (self.maxlength - n, 0))
            schapter = np.pad(schapter, (self.maxlength - n, 0))
            user_feats = [[0]*len(user_feats[0])]*(self.maxlength  - n)+list(user_feats[:])
            user_feats = np.array(user_feats, np.float)
            responses = np.append(2, labels[:-1])
            labels = np.pad(labels, (self.maxlength - n, 0))
            responses = np.pad(responses, (self.maxlength - n, 0), mode='constant', constant_values = 2)
        
        new_features = []
        count = 0
        for f in user_feats:
          temp = list(f)
#           temp.extend(term_encode)
          new_features.append(temp)
          count += 1
        user_feats = new_features
        
        return question_id, chapter, schapter, responses, np.array(user_feats), labels 

In [30]:
NUMERIC_FEATURE =  ['age',
            'exclClassCumGPA',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            "day's_available_flashcards", 
            'days_offset', 
            'prev_time_elapsed',
             'time_lag']
 
for f in NUMERIC_FEATURE:
  m = data[f].mean()
  std = data[f].std()
  data[f] = (data[f] - m)/std

In [31]:
#group data based on user id
group = data.groupby(['user_id.x']).apply(lambda r: (
                r['question_id'],
                r['chapter_id'],
                r['sub_chapter_id'],
                r['term'],
                np.array([
#                     r['STDNT_SEX_CD'],r['STDNT_CTZN_STAT_CD'], r['international'], 
#                   r['White'],r['Asian'],r['NonWhiteOrAsian'], r['age'],r['exclClassCumGPA'],
#                 r['Freshman'], r['Junior'], r['Sophomore'], r['Senior'],
#                 r['termCreditsGPA'], r['termCreditsNoGPA'],
#                 r['athlete_1'], r['honorsPro'],
#                 r['LSA'], r['programBusiness'], r['programEngineering'], 
#                 r['programInformation'], r['programOther'],
#                 r['HSCalculusTaken'],  r['highSchoolGPA'], 
#                 r['majorsCount'], r['minorsCount'],
#                 r['PREV_TERM_CUM_GPA'], 
#                 r['parentsGraduateEdu'], r['minorityGroup'],
#                 r["day's_available_flashcards"],
#                 r['days_offset'],
                r['prev_time_elapsed'],
                r['time_lag']
              ]).transpose(),
                r['answered_correctly'],
                ))

In [32]:
patience = 5

In [33]:
# remove students who don't have make any interactions with the tool
toRemove = []
for index in group.index:
  if len(group[index][0]) <= 10:
    toRemove.append(index)
group = group.drop(index=toRemove)

In [34]:
X = np.array(group.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
train_accs = list()
val_losses = list()
val_aucs = list()
val_accs = list()
test_losses = list()
test_aucs = list()
test_accs = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train = PRACTICE_DATASET(group[users_train])
    valid = PRACTICE_DATASET(group[users_val])
    test = PRACTICE_DATASET(group[users_test])
    train_dataloader = DataLoader(train, batch_size=64, shuffle=True, num_workers=8)
    val_dataloader = DataLoader(valid, batch_size=64, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(test, batch_size=64, shuffle=True, num_workers=8)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    saint = SAINTModel(n_skill, n_part)
    epochs = 100
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(saint.parameters(), betas=(0.9, 0.999), lr = 0.0005, eps=1e-8)
    saint.to(device)
    criterion.to(device)
    
    def train_epoch(model=saint, train_iterator=train_dataloader, optim=optimizer, criterion=criterion, device=device):
        model.train()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(train_iterator)
        for item in tbar:
            question_id = item[0].to(device).long()
            chapter = item[1].to(device).long()
            schapter = item[2].to(device).long()
            responses = item[3].to(device).long()
            user_feats = item[4].to(device).float()
            label = item[5].to(device).float()            
            target_mask = (question_id!=0)
            optim.zero_grad()
            output = model(question_id, chapter, schapter, responses, user_feats)
            output = torch.reshape(output, label.shape)

            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            loss.backward()
            optim.step()
            train_loss.append(loss.item())
            pred = (torch.sigmoid(output) >= 0.5).long()

            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('loss - {:.4f}'.format(loss))
        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.mean(train_loss)

        return loss, acc, auc
   

    def val_epoch(model=saint, val_iterator=test_dataloader, 
              criterion=criterion, device=device):
        model.eval()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(val_iterator)
        for item in tbar:
            question_id = item[0].to(device).long()
            chapter = item[1].to(device).long()
            schapter = item[2].to(device).long()
            responses = item[3].to(device).long()
            user_feats = item[4].to(device).float()
            label = item[5].to(device).float()            
            target_mask = (question_id!=0)
            with torch.no_grad():
                output = model(question_id, chapter, schapter, responses, user_feats)

            output = torch.reshape(output, label.shape)
            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            train_loss.append(loss.item())

            pred = (torch.sigmoid(output) >= 0.5).long()
            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('valid loss - {:.4f}'.format(loss))

        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.average(train_loss)

        return loss, acc, auc
    
    MIN_VAL = 1000000000
    count = 0
    print('----------------------------------------------------------------------------')
    for epoch in range(epochs):
        train_loss, train_acc, train_auc = train_epoch(model=saint, device=device)
        print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, train_loss, train_acc, train_auc))
        val_loss, val_acc, val_auc = val_epoch(model=saint, val_iterator= val_dataloader, device=device)
        print("epoch - {} val_loss - {:.2f} val acc - {:.3f} val auc - {:.3f}".format(epoch, val_loss, val_acc, val_auc))
        if val_loss < MIN_VAL:
            count = 0
            MIN_VAL = val_loss
        else:
            count += 1

        if count == patience:
            print('Val Loss does not improve for {} consecutive epochs'.format(patience))
            break
    test_loss, test_acc, test_auc = val_epoch(model=saint, device=device)
    print("epoch - {} test_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, test_loss, test_acc, test_auc))
    test_losses.append(test_loss)
    test_aucs.append(test_auc)
    test_accs.append(test_acc)
    train_aucs.append(train_auc)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

  0%|          | 0/14 [00:00<?, ?it/s]

----------------------------------------------------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 0 train_loss - 0.66 acc - 0.701 auc - 0.522


  0%|          | 0/14 [00:00<?, ?it/s]

...

epoch - 15 train_loss - 0.42 acc - 0.798 auc - 0.846


  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 15 val_loss - 0.55 val acc - 0.718 val auc - 0.750
Val Loss does not improve for 5 consecutive epochs


valid loss - 0.4123: 100%|██████████| 2/2 [00:00<00:00,  4.89it/s]

epoch - 15 test_loss - 0.42 acc - 0.804 auc - 0.833


In [35]:
print("test avg loss: ", np.mean(test_losses), np.std(test_losses))
print("test avg acc: ", np.mean(test_accs), np.std(test_accs))
print("test avg auc: ", np.mean(test_aucs), np.std(test_aucs))

test avg loss:  0.40710650086402894 0.00958387464644376
test avg acc:  0.8119897418575892 0.008455992806483117
test avg auc:  0.8309953008051542 0.006034758388840082


In [36]:
print("train avg loss: ", np.mean(train_losses), np.std(train_losses))
print("train avg acc: ", np.mean(train_accs), np.std(train_accs))
print("train avg auc: ", np.mean(train_aucs), np.std(train_aucs))

train avg loss:  0.40161789272512705 0.009704411945629246
train avg acc:  0.8057823254943568 0.005184044282737314
train avg auc:  0.8584211896069398 0.008623035643522068
